## Flight Delay Claims Prediction – Modeling

This notebook contains the source code of the modeling process of the Flight Delay Claims Prediction project.

### Import packages

In [1]:
import requests

import numpy as np
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

from joblib import dump, load

### Load data

The CSV file is first loaded as a pandas data frame, which allows it to be read and manipulated easily. We can show the first 10 lines of the CSV file to examine the schema and some sample rows.

In [2]:
flights = pd.read_csv('../data/flight_delays_data.csv', parse_dates=['flight_date'])

### Data cleaning

The thought process of the following data cleaning procedures has been covered in the EDA notebook.

#### Fill in missing airline

In [3]:
flights.loc[flights.Airline.isnull(), 'Airline'] = flights.loc[flights.Airline.isnull(), 'flight_no'].str[:2]

#### Separate canceled flights

In [4]:
flights['is_cancelled'] = np.where(flights.delay_time == 'Cancelled', 1, 0)
flights.loc[flights.delay_time == 'Cancelled', 'delay_time'] = max(3, float(flights.loc[flights.delay_time != 'Cancelled', 'delay_time'].max()))
flights.delay_time = flights.delay_time.astype(float)

### Feature engineering

The thought process of the following feature engineering procedures has been covered in the EDA notebook.

#### Year, month, day, and day of week

In [5]:
flights['flight_date_year'] = flights.flight_date.dt.year
flights['flight_date_month'] = flights.flight_date.dt.month
flights['flight_date_day'] = flights.flight_date.dt.day
flights['flight_date_dow'] = flights.flight_date.dt.dayofweek

flights.Week = flights.Week.astype('category')
flights.flight_date_year = flights.flight_date_year.astype('category')
flights.flight_date_month = flights.flight_date_month.astype('category')
flights.flight_date_day = flights.flight_date_day.astype('category')
flights.flight_date_dow = flights.flight_date_dow.astype('category')

#### Hong Kong public holidays

In [6]:
flight_years = flights.flight_date_year.unique()

In [7]:
def get_hk_holiday_data(years):
    public_holidays = list()

    for year in years:
        holiday_url = f'https://www.gov.hk/en/about/abouthk/holiday/{year}.htm'
        r = requests.get(holiday_url, headers={'User-Agent': 'Mozilla/5.0'})
        holiday_dates = pd.read_html(r.text, skiprows=1)[0][1].apply(lambda x: f'{x} {year}')
        holiday_dates = pd.to_datetime(holiday_dates, infer_datetime_format=True)
        public_holidays.extend(holiday_dates)
    
    return public_holidays

In [8]:
public_holidays = get_hk_holiday_data(flight_years)
flights['is_public_holiday'] = np.where(flights.flight_date.isin(public_holidays), 1, 0)

#### Hong Kong weather data

In [9]:
def get_weather_data(years):
    weather_list = list()

    for year in years:
        weather_url = f'https://www.hko.gov.hk/cis/dailyExtract/dailyExtract_{year}'
        r = requests.get(weather_url, params={'y': year})
        weather_data = r.json()['stn']['data']

        for elem_month in weather_data:
            month = elem_month['month']
            day_data = elem_month['dayData'][:-2]
            for elem_day in day_data:
                day = elem_day[0]
                mean_pressure = float(elem_day[1])
                mean_temp = float(elem_day[3])
                mean_dew_point = float(elem_day[5])
                mean_humidity = float(elem_day[6])
                mean_cloud = float(elem_day[7])
                mean_rainfall = float(elem_day[8]) if elem_day[8] != 'Trace' else 0.0
                weather_list.append({'flight_date': pd.to_datetime(f'{year}-{month:02}-{day}'),
                                     'mean_pressure': mean_pressure, 'mean_temp': mean_temp, 'mean_dew_point': mean_dew_point,
                                     'mean_humidity': mean_humidity, 'mean_cloud': mean_cloud, 'total_rainfall': mean_rainfall})

    weather_df = pd.DataFrame(weather_list)
    return weather_df

In [10]:
weather_df = get_weather_data(flight_years)
flights = pd.merge(flights, weather_df, on='flight_date', how='left')

#### Clean up

In [11]:
# Convert departure hour to category
flights.std_hour = flights.std_hour.astype('category')

# Drop unecessary flight_date and flight_id column
flights = flights.drop(columns=['flight_date', 'flight_id'])

Since it is likely that `delay_time`, `is_cancelled` and `is_claim` would not exist in the hidden set, and the fact that the goal of the project is to predict `delay_time` and derive `is_claim`, `is_cancelled` and `is_claim` will be removed from the training set. Since the `delay_time` column contains enough information for us to derive `is_claim`, we can safely proceed without `is_cancelled`.

In [12]:
flights = flights.drop(columns=['is_cancelled', 'is_claim'])

In [13]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 899114 entries, 0 to 899113
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   flight_no          899114 non-null  object  
 1   Week               899114 non-null  category
 2   Departure          899114 non-null  object  
 3   Arrival            899114 non-null  object  
 4   Airline            899114 non-null  object  
 5   std_hour           899114 non-null  category
 6   delay_time         899114 non-null  float64 
 7   flight_date_year   899114 non-null  category
 8   flight_date_month  899114 non-null  category
 9   flight_date_day    899114 non-null  category
 10  flight_date_dow    899114 non-null  category
 11  is_public_holiday  899114 non-null  int64   
 12  mean_pressure      899114 non-null  float64 
 13  mean_temp          899114 non-null  float64 
 14  mean_dew_point     899114 non-null  float64 
 15  mean_humidity      899114 non-null

In [14]:
flights_features = flights.drop(columns=['delay_time'])
cat_cols_idx = [flights_features.columns.get_loc(c) for c in list(flights_features.select_dtypes(exclude=[np.number]).columns)]
num_cols_idx = [flights_features.columns.get_loc(c) for c in list(flights_features.select_dtypes(include=[np.number]).columns)]

### Modeling

I decided to proceed with the modeling stage using a PyTorch 2-layer fully connected neural network model.

The following code is modified from this notebook by Yannet Interian: https://github.com/yanneta/deep-learning-with-pytorch/blob/master/lesson2-tabular.ipynb

#### Split train and validation sets

In [15]:
def split_dataset(dataset, target, seed=1):
    """
    Splits dataset into training and validation sets
    """
    X = np.array(dataset.drop(columns=[target]))
    y = np.array(dataset[target])
    train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state=seed)
    return train_X, valid_X, train_y, valid_y

In [16]:
train_X, valid_X, train_y, valid_y = split_dataset(flights, 'delay_time')

#### Feature encoding

In [17]:
def encode_cat_variables(x, help_dict = None):
    """
    Encodes a categorical variable.
    The index 0 is left for values not in training.
    """
    uniqs = np.unique(x)
    if help_dict is None: help_dict = {v: k + 1 for k, v in enumerate(uniqs)}
    levels = len(help_dict.keys()) + 1
    x_t = np.array([help_dict.get(x_i, 0) for x_i in x])
    return x_t, help_dict, levels

In [18]:
def transform_dataset(train_X, valid_X, cat_ind, num_ind):
    """
    Transform the dataset by encoding features.
    """    
    train_X_cat = train_X[:, cat_ind]
    train_X_num = train_X[:, num_ind]
    
    valid_X_cat = valid_X[:, cat_ind]
    valid_X_num = valid_X[:, num_ind]
    
    # Transform numerical variables
    scaler = StandardScaler()
    train_X_num = scaler.fit_transform(train_X_num)
    valid_X_num = scaler.transform(valid_X_num)
        
    # Transform categorical variables
    level_arr = [0] * train_X_cat.shape[1]
    help_dict = list()
    for i in range(train_X_cat.shape[1]):
        x, help_arr, levels = encode_cat_variables(train_X_cat[:, i])
        train_X_cat[:, i] = x
        level_arr[i] = levels
        help_dict.append(help_arr)
        x, _, _ = encode_cat_variables(valid_X_cat[:, i], help_arr)
        valid_X_cat[:, i] = x
    
    train_X_cat = np.array(train_X_cat).astype(int)
    valid_X_cat = np.array(valid_X_cat).astype(int)
    
    return (train_X_cat, train_X_num, valid_X_cat, valid_X_num), level_arr, scaler, help_dict

In [19]:
(train_X_cat, train_X_num, valid_X_cat, valid_X_num), level_arr, scaler, help_dict = transform_dataset(train_X, valid_X, cat_cols_idx, num_cols_idx)

In [20]:
# Save scaler and help_dict
model_files_dir = '../models/helper_files'
dump(scaler, f'{model_files_dir}/scaler.joblib')
dump(help_dict, f'{model_files_dir}/help_dict.joblib') 

['../models/helper_files/help_dict.joblib']

#### Dataset and Dataloader

In [21]:
class TabularDataSet(Dataset):
    """
    Dataset object for tabular data.
    """
    def __init__(self, X_cat, X_num, Y):
        self.X_cat = X_cat
        self.X_num = X_num
        self.Y = Y

    def __getitem__(self, index):
        return self.X_cat[index], self.X_num[index], self.Y[index]

    def __len__(self):
        return len(self.Y)

In [22]:
batch_size = 10000
train_ds = TabularDataSet(train_X_cat, train_X_num, train_y)
valid_ds = TabularDataSet(valid_X_cat, valid_X_num, valid_y)

In [23]:
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

#### Model definition

In [24]:
class TabularNet(nn.Module):
    """
    2 layer fully connected neural network model.
    """
    def __init__(self, num_cont, num_cat, level_arr, hidden_dim=1000, hidden_dim2=1000):
        super(TabularNet, self).__init__()
        in_dim = num_cont + 2 * num_cat
        self.embs = nn.ModuleList([nn.Embedding(level_arr[i], 2) for i in range(len(level_arr))])
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim2)
        self.linear1 = nn.Linear(in_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim2)
        self.linear3 = nn.Linear(hidden_dim2, 1)
        self.dropout = nn.Dropout(0.2)
                                  
    def forward(self, x_cat, x_cont):
        x_cat = [self.embs[i](x_cat[:,i]) for i in range(x_cat.size(1))]
        x_cat = torch.cat(x_cat, dim=1)
        x_cat = self.dropout(x_cat)
        x = torch.cat([x_cont, x_cat], dim=1)
        x = self.bn1(F.relu(self.linear1(x)))
        x = self.dropout(x)
        x = self.bn2(F.relu(self.linear2(x)))
        return self.linear3(x)

#### Model training

In [25]:
# Helper function to save and retrieve trained models
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [26]:
# Helper functions to compute the consine learning rates
def cosine_segment(start_lr, end_lr, iterations):
    i = np.arange(iterations)
    c_i = 1 + np.cos(i*np.pi/iterations)
    return end_lr + (start_lr - end_lr)/2 * c_i

def get_cosine_triangular_lr(max_lr, iterations, div_start=5, div_end=5):
    min_start, min_end = max_lr/div_start, max_lr/div_end
    iter1 = int(0.3*iterations)
    iter2 = iterations - iter1
    segs = [cosine_segment(min_start, max_lr, iter1), cosine_segment(max_lr, min_end, iter2)]
    return np.concatenate(segs)

In [27]:
def update_optimizer(optimizer, lr):
    for i, param_group in enumerate(optimizer.param_groups):
        param_group['lr'] = lr

In [28]:
def train_model(model, train_dl, valid_dl, optimizer, max_lr=0.05, epochs=100):
    iterations = epochs * len(train_dl)
    idx = 0
    best_val_r2 = 0
    lrs = get_cosine_triangular_lr(max_lr, iterations)
    
    for t in range(epochs):
        model.train()
        total_loss = 0
        total = 0
        for x1, x2, y in train_dl:
            update_optimizer(optimizer, lrs[idx])
            x1 = x1.long() #.cuda()
            x2 = x2.float() #.cuda()
            y = y.unsqueeze(1).float() #.cuda()
            y_hat = model(x1, x2)
            loss = F.mse_loss(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * y.size(0)
            total += y.size(0)
            idx += 1
        val_loss, val_r2 = val_metrics(model, valid_dl)
        print(f'Train loss: {total_loss/total:.3f} \t Valid loss: {val_loss:.3f} \t Valid R2: {val_r2:.3f}')  
        
        # Saved the best trained model
        if best_val_r2 < val_r2:
            best_val_r2 = val_r2
            path = f'../models/model_flights_r2_{int(100 * val_r2):02}.pth'
            save_model(model, path)
            print(path)
            
    print(f'Best valid r2: {best_val_r2:.3f}')
    return best_val_r2

In [29]:
def val_metrics(model, valid_dl):
    model.eval()
    total = 0
    sum_loss = 0
    y_hat = []
    ys = []
    
    for x1, x2, y in valid_dl:
        batch = y.shape[0]
        y = y.unsqueeze(1).float()
        out = model(x1.long(), x2.float()) #.cuda()
        loss = F.mse_loss(out, y) #.cuda(
        sum_loss += batch * (loss.item())
        total += batch
        y_hat.append(out.detach().numpy()) # .cpu().
        ys.append(y)
    
    y_hat = np.vstack(y_hat)
    ys = np.vstack(ys)
    r2 = r2_score(ys, y_hat)
    return sum_loss/total, r2

In [30]:
num_cont = train_X_num.shape[1]
num_cat = train_X_cat.shape[1]
model = TabularNet(num_cont, num_cat, level_arr)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=1e-5)
best_val = train_model(model, train_dl, valid_dl, optimizer, epochs=100)

Train loss: 6.067 	 Valid loss: 3.895 	 Valid R2: 0.037
../models/model_flights_r2_03.pth
Train loss: 3.847 	 Valid loss: 3.774 	 Valid R2: 0.066
../models/model_flights_r2_06.pth
Train loss: 3.770 	 Valid loss: 3.687 	 Valid R2: 0.088
../models/model_flights_r2_08.pth
Train loss: 3.675 	 Valid loss: 3.569 	 Valid R2: 0.117
../models/model_flights_r2_11.pth
Train loss: 3.483 	 Valid loss: 3.220 	 Valid R2: 0.203
../models/model_flights_r2_20.pth
Train loss: 3.291 	 Valid loss: 3.129 	 Valid R2: 0.226
../models/model_flights_r2_22.pth
Train loss: 3.189 	 Valid loss: 3.009 	 Valid R2: 0.256
../models/model_flights_r2_25.pth
Train loss: 3.126 	 Valid loss: 2.981 	 Valid R2: 0.263
../models/model_flights_r2_26.pth
Train loss: 3.084 	 Valid loss: 2.924 	 Valid R2: 0.277
../models/model_flights_r2_27.pth
Train loss: 3.047 	 Valid loss: 2.971 	 Valid R2: 0.265
Train loss: 3.027 	 Valid loss: 2.884 	 Valid R2: 0.287
../models/model_flights_r2_28.pth
Train loss: 2.962 	 Valid loss: 2.873 	 Vali